In [ ]:
import os
import pandas as pd
from datetime import datetimez

In [ ]:
# AWS S3 Functions
def files_s3_bucket(Bucketname, Folder, Extension):
   """
   Returns the list of files in an S3 bucket filtered by folder and file extension.
   Args:
       Bucketname (str): Name of the S3 bucket, e.g. 'voiceofvehicle'
       Folder (str): Prefix/folder inside the bucket, e.g. '0_TEST/'
       Extension (str): File extension to filter by, e.g. '.csv'
   Returns:
       list[str]: List of object keys (paths) matching the folder and extension.
   """
   s3 = boto3.client('s3')
   response = s3.list_objects_v2(Bucket=Bucketname, Prefix=Folder)


   object_keys = [
       obj['Key'] for obj in response.get('Contents', [])
       if obj['Key'] != Folder and obj['Key'].endswith(Extension)
   ]
   return object_keys

def list_all_s3_folders(bucket_name, prefix=''):
   """
   Recursively list all folders (prefixes) in an S3 bucket.
   Args:
       bucket_name (str): The name of the S3 bucket.
       prefix (str): Optional base path to start from (e.g. 'data/')
   Returns:
       list[str]: A sorted list of unique folder paths.
   """
   s3 = boto3.client('s3')
   paginator = s3.get_paginator('list_objects_v2')
   # Store found folder paths
   folders = set()
   for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
       for obj in page.get('Contents', []):
           key = obj['Key']
           if '/' in key:
               parts = key.split('/')[:-1]  # drop filename, keep folders
               for i in range(1, len(parts) + 1):
                   folders.add('/'.join(parts[:i]) + '/')
   return sorted(folders)

def preserve_acl_before_upload(bucket, key):
   import boto3
   s3 = boto3.client('s3')
   try:
       return s3.get_object_acl(Bucket=bucket, Key=key)
   except ClientError as e:
       if e.response['Error']['Code'] == 'NoSuchKey':
           return None
       else:
           raise
def preserve_acl_before_upload2(s3, bucket, key):
   try:
       return s3.get_object_acl(Bucket=bucket, Key=key)
   except ClientError as e:
       if e.response['Error']['Code'] == 'NoSuchKey':
           return None
       else:
           raise

def reapply_acl_after_upload(bucket, key, acl):
   import boto3
   s3 = boto3.client('s3')
   if acl:
       s3.put_object_acl(
           Bucket=bucket,
           Key=key,
           AccessControlPolicy={
               'Grants': acl['Grants'],
               'Owner': acl['Owner']
           }
       )
def reapply_acl_after_upload2(s3, bucket, key, acl):
   if acl:
       s3.put_object_acl(Bucket=bucket, Key=key, AccessControlPolicy=acl)

In [ ]:
# Define the folder path containing .xls files
folder_path ="W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Raw_Data/"

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xls"):# Check if the file is an .xls file
        file_path = os.path.join(folder_path, file_name) # Construct the full file path
        df = pd.read_excel(file_path)# Read the Excel file into a DataFrame
        dfs.append(df)# Append the DataFrame to the lis
        week_number = pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y').week
        month=pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y').month
        month=pd.to_datetime(month, format='%m').strftime('%B')
        date=pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y').date()
        date_datetime=pd.to_datetime(date)
    
merged_df = pd.concat(dfs, ignore_index=True)# Merge all DataFrames in the list into a single DataFrame

# Define the renaming mapping
rename_map = {
    "Comment": "0 Comment",
    "Problem Statement - Anecdote": "1 Problem Statement - Anecdote",
    "Immediate Action": "2 Immediate Action",
    "Problem Definition and Severity": "3 Problem Definition and Severity",
    "Short Term Fix": "4 Short Term Fix",
    "Root Cause": "5 Root Cause",
    "Long term fix technical development and validation": "6 Long term fix technical development and validation",
    "Long term fix deployment": "7 Long term fix deployment",
    "Lessons Learned": "8 Lessons Learned"}

# Apply renaming to the "NextStepAction" column
merged_df['NextStepAction'] = merged_df['NextStepAction'].map(rename_map)

# Print the merged DataFrame
merged_df.head(4)

print(date_datetime)

df=merged_df

2024-05-30 00:00:00


In [ ]:
# Convert 'LastUpdatedDate' and 'CreateDate' columns to datetime if they're not already in datetime format
df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'])
df['CreateDate'] = pd.to_datetime(df['CreateDate'])

# Sort the dataframe by 'ShortId', 'NextStepAction', and 'LastUpdatedDate'
df = df.sort_values(by=['ShortId', 'LastUpdatedDate'])

# Initialize a new column to store the duration in the same status as integers (days only)
df['TimeInSameStatus'] = 0

# Initialize a new column to store the age of the ticket based on CreateDate
df['Age'] = (date_datetime - df['CreateDate']).dt.days

# Initialize a new column to indicate if a ticket is in its last status
df['IsLastStatus'] = False

# Group by 'ShortId' and iterate through each group to find the last status
for short_id, group in df.groupby('ShortId'):
    last_index = group.index[-1]
    df.at[last_index, 'IsLastStatus'] = True
    
# Define a function to determine the fix type based on NextStepAction
def determine_fix_type(next_step_action):
    if next_step_action in ["3 Problem Definition and Severity", "4 Short Term Fix", "5 Root Cause"]:
        return "InShortTermFix"
    else:
        return "InLongTermFix"

# Initialize the 'FixType' column with default value
df['FixType'] = ""

# Group by 'ShortId' and iterate through each group
for short_id, group in df.groupby('ShortId'):
    last_status = group['NextStepAction'].iloc[-1]
    
    # Determine the fix type for the entire group
    fix_type = determine_fix_type(last_status)
    
    # Set the 'FixType' column for all rows in the group
    df.loc[df['ShortId'] == short_id, 'FixType'] = fix_type

# Assuming df is your pandas DataFrame
df = df.sort_values(by=['ShortId', 'LastUpdatedDate'])

# Initialize a new column to store the time differences
df['TimeInSameStatus'] = None

# Group by 'ShortId' and iterate through each group
for short_id, group in df.groupby('ShortId'):
    # Get the last status of the group
    last_status = group['Status'].iloc[-1]
    # Check if the last status is 'Resolved'
    if last_status == 'Resolved':
        # Calculate time difference for the last status to the first one
        first_date = group['LastUpdatedDate'].iloc[0]
        last_date = group['LastUpdatedDate'].iloc[-1]
        last_status_diff = (last_date - first_date).days
        # Store time difference in the original DataFrame for the last status
        df.loc[df['ShortId'] == short_id, 'TimeInSameStatus'] = last_status_diff
        # Mark all tickets in the group as 'Resolved'
        df.loc[df['ShortId'] == short_id, 'Status'] = 'Resolved'
    else:
        # Filter the DataFrame for the current shortId
        filtered_df = df[df['ShortId'] == short_id]
        # Check if the first status is "Open"
        if filtered_df['Status'].iloc[0] == 'Open':
            list_date = []
            list_status = []
            for step in filtered_df['NextStepAction'].unique():
                filtered_df_status = filtered_df[filtered_df['NextStepAction'] == step]
                min_date_status = filtered_df_status['LastUpdatedDate'].min()
                list_date.append(pd.to_datetime(min_date_status))  # Convert date string to datetime object
                list_status.append(step)
            # Calculate time differences between consecutive dates for each status
            time_diffs = []
            for i in range(1, len(list_date)):
                time_diff = (list_date[i] - list_date[i-1]).days
                time_diffs.append((list_status[i-1], time_diff))
            # Calculate time difference for the last status to today
            if list_date:
                if len(list_date) == 1:  # Check if there's only one status
                    last_status_diff = (date_datetime - list_date[0]).days
                    # Store time difference in the original DataFrame for the first status
                    df.loc[(df['ShortId'] == short_id) & (df['NextStepAction'] == list_status[0]), 'TimeInSameStatus'] = last_status_diff
                else:
                    last_status_diff = (date_datetime - list_date[-1]).days
                    # Store time difference in the original DataFrame for the last status
                    df.loc[(df['ShortId'] == short_id) & (df['NextStepAction'] == list_status[-1]), 'TimeInSameStatus'] = last_status_diff
            # Store time differences for consecutive statuses
            for status, time_diff in time_diffs:
                # Store time difference in the original DataFrame for each status
                df.loc[(df['ShortId'] == short_id) & (df['NextStepAction'] == status), 'TimeInSameStatus'] = time_diff
        else:
            list_date = []
            list_status = []
            for step in filtered_df['NextStepAction'].unique():
                filtered_df_status = filtered_df[filtered_df['NextStepAction'] == step]
                min_date_status = filtered_df_status['LastUpdatedDate'].min()
                list_date.append(pd.to_datetime(min_date_status))  # Convert date string to datetime object
                list_status.append(step)
            # Calculate time differences between consecutive dates for each status
            time_diffs = []
            for i in range(1, len(list_date)):
                time_diff = (list_date[i] - list_date[i-1]).days
                time_diffs.append((list_status[i-1], time_diff))
            # Calculate time difference for the last status to the first one
            if list_date:
                last_status_diff = (list_date[-1] - list_date[0]).days
                # Store time difference in the original DataFrame for the last status
                df.loc[(df['ShortId'] == short_id) & (df['NextStepAction'] == list_status[-1]), 'TimeInSameStatus'] = last_status_diff
            # Store time differences for consecutive statuses
            for status, time_diff in time_diffs:
                # Store time difference in the original DataFrame for each status
                df.loc[(df['ShortId'] == short_id) & (df['NextStepAction'] == status), 'TimeInSameStatus'] = time_diff

# Ensure all time differences are positive
df['TimeInSameStatus'] = df['TimeInSameStatus'].fillna(0).abs()

In [ ]:
#Export data

# Define the path where you want to save the CSV file
file_path = "W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Raw_Data/Merged_Data/"

# Define the filename with last's week number from folder and the path
file_name = f"{file_path}Raw_Data_Calculation_week_{week_number}.csv"

file_name_2 = f"{file_path}Data_Merged_with_Calculations.csv"
# Export the pivot table DataFrame to a CSV file at the specified path
df.to_csv(file_name, index=False)
df.to_csv(file_name_2, index=False)

print(f"CSV file exported successfully to: {file_name}")
print(f"CSV file exported successfully to: {file_name_2}")

CSV file exported successfully to: W:/Shared With Me/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Raw_Data/Merged_Data/Raw_Data_Calculation_week_22.csv
CSV file exported successfully to: W:/Shared With Me/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Raw_Data/Merged_Data/Data_Merged_with_Calculations.csv


In [ ]:
#Calcuations of Running totals

# List of values to filter by
filter_list = ['3 Problem Definition and Severity', '4 Short Term Fix', '5 Root Cause','6 Long term fix technical development and validation','7 Long term fix deployment', '8 Lessons Learned']

# Filter the DataFrame
filtered_df = df[df['NextStepAction'].isin(filter_list)]

pivot_table = filtered_df.pivot_table(values='TimeInSameStatus',
                             index=['Status','Title','NextStepAction' ],
                             aggfunc='max',
                             fill_value=0)

running_totals = pivot_table.groupby(level='Title').cumsum(axis=0)
running_totals=running_totals.assign(**{"Week_number":week_number})

#Export weekly running
#file_path_VCM_Metrics_weekly = f"C:/Users/{user}/Desktop/Python/Dashboard/Weekly_KPI/VCM_Metrics/"
file_path_VCM_Metrics_weekly = "W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Weekly_KPI/VCM_Metrics/"
file_name_VCM_Metrics_weekly = f"{file_path_VCM_Metrics_weekly}VCM_Metrics_Wk_{week_number}.csv"
running_totals.to_csv(file_name_VCM_Metrics_weekly, index=True)


#Export Actual
#file_path_VCM_Metrics_actual = f"C:/Users/{user}/Desktop/Python/Dashboard/Weekly_KPI/VCM_Metrics/Merged/"
file_path_VCM_Metrics_actual = f"W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Weekly_KPI/VCM_Metrics/Merged/"

file_name_VCM_Metrics_actual = f"{file_path_VCM_Metrics_actual}VCM_Metrics_Actual.csv"
running_totals.to_csv(file_name_VCM_Metrics_actual, index=True)

# Loop through each file in the folder
dfs = []

for file_name in os.listdir(file_path_VCM_Metrics_weekly):
    # Check if the file is an .xls file
    if file_name.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(file_path_VCM_Metrics_weekly, file_name)
        # Read the Excel file into a DataFrame
        df = pd.read_csv(file_path)
        
        dfs.append(df)
        
# Merge all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=False)
merged_df.to_csv(f"W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Weekly_KPI/VCM_Metrics/Merged/VCM_Metrics_Merged.csv", index=False)

In [ ]:
#Calculation of Goal Metric
df=running_totals.reset_index()

filter_list = ['3 Problem Definition and Severity', '4 Short Term Fix', '5 Root Cause']

# Filter the DataFrame
filtered_df = df[df['NextStepAction'].isin(filter_list)]


filtered_df= pd.pivot_table(filtered_df, index=["Title","Status"],values="TimeInSameStatus", aggfunc='max')
filtered_df=filtered_df.reset_index()

interval = 90
min_interval = 0
max_interval = filtered_df['TimeInSameStatus'].max()

bins = range(min_interval, max_interval + interval, interval)

# Cut the data into intervals
filtered_df['TimeInSameStatus'] = pd.cut(filtered_df['TimeInSameStatus'], bins)

# Create pivot table to count occurrences of each interval
Pivot_Groups = pd.pivot_table(filtered_df, index=["Status","TimeInSameStatus"],values="Title", aggfunc=pd.Series.nunique)

# Calculate percentage

total_counts = Pivot_Groups.groupby('Status')['Title'].transform('sum')
Pivot_Groups['Percentage'] = Pivot_Groups['Title'] / total_counts * 100


Pivot_Groups=Pivot_Groups.assign(**{"Week_number":week_number})

#Export weekly running

#file_path_VCM_weekly_Goal = f"C:/Users/{user}/Desktop/Python/Dashboard/Weekly_KPI/VCM_Metrics/Goal_Metric/"
file_path_VCM_weekly_Goal = "W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Weekly_KPI/VCM_Metrics/Goal_Metric/"
file_name_VCM_weekly_Goal = f"{file_path_VCM_weekly_Goal}VCM_Goal_Metric_Wk_{week_number}.csv"

#file_path_VCM_weekly_Goal_merged= f"C:/Users/{user}/Desktop/Python/Dashboard/Weekly_KPI/VCM_Metrics/Goal_Metric/Merged/"
file_path_VCM_weekly_Goal_merged= "W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Weekly_KPI/VCM_Metrics/Goal_Metric/Merged/"
file_name_VCM_weekly_Goal_merged=f"{file_path_VCM_weekly_Goal_merged}VCM_Goal_Metric_Merged.csv"
Pivot_Groups.to_csv(file_name_VCM_weekly_Goal, index=True)

# Loop through each file in the folder
dfs = []

for file_name in os.listdir(file_path_VCM_weekly_Goal):
    # Check if the file is an .xls file
    if file_name.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(file_path_VCM_weekly_Goal, file_name)
        # Read the Excel file into a DataFrame
        df = pd.read_csv(file_path)
        
        dfs.append(df)
        

# Merge all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=False)
merged_df.to_csv(file_name_VCM_weekly_Goal_merged, index=False)

print ('Successful' )

Successful


In [ ]:
# Subir archivos csv a S3 mantieniendo ACL:
file="W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/"

upload_file_to_s3_with_acl(
    file_path = f"{file}Raw_Data/Merged_Data/Data_Merged_with_Calculations.csv",
    bucket_name="voiceofvehicle",
    key="dashboardvcm/Data_Merged_with_Calculations.csv"
)

upload_file_to_s3_with_acl(
    file_path = f"{file}Weekly_KPI/VCM_Metrics/Merged/VCM_Metrics_Actual.csv",
    bucket_name="voiceofvehicle",
    key="dashboardvcm/VCM_Metrics_Actual.csv"
)

upload_file_to_s3_with_acl(
    file_path= f"{file}Weekly_KPI/VCM_Metrics/Goal_Metric/Merged/VCM_Goal_Metric_Merged.csv",
    bucket_name="voiceofvehicle",
    key="dashboardvcm/VCM_Goal_Metric_Merged.csv"

For TFE screenshots

In [ ]:
import os
import pandas as pd

# Define the folder path containing .xls files
folder_path = "W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Raw_Data/"

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xls"):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        dfs.append(df)
        week_number = pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y').week
        month = pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y').month
        month = pd.to_datetime(month, format='%m').strftime('%B')
        date = pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y').date()
        date_datetime = pd.to_datetime(date)

# Merge all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Define the renaming mapping for standardizing phase names
rename_map = {
    "Comment": "0 Comment",
    "Problem Statement - Anecdote": "1 Problem Statement - Anecdote",
    "Immediate Action": "2 Immediate Action",
    "Problem Definition and Severity": "3 Problem Definition and Severity",
    "Short Term Fix": "4 Short Term Fix",
    "Root Cause": "5 Root Cause",
    "Long term fix technical development and validation": "6 Long term fix technical development and validation",
    "Long term fix deployment": "7 Long term fix deployment",
    "Lessons Learned": "8 Lessons Learned"
}

# Apply renaming to the "NextStepAction" column
merged_df['NextStepAction'] = merged_df['NextStepAction'].map(rename_map)

# Convert date columns to datetime format
merged_df['LastUpdatedDate'] = pd.to_datetime(merged_df['LastUpdatedDate'])
merged_df['CreateDate'] = pd.to_datetime(merged_df['CreateDate'])

# Sort chronologically
merged_df = merged_df.sort_values(by=['ShortId', 'LastUpdatedDate'])

# Save the final cleaned and unified DataFrame
df = merged_df


In [ ]:
#Bloque 1: Conversión de fechas, ordenación y columnas auxiliares
# Convert columns to datetime type
df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'])
df['CreateDate'] = pd.to_datetime(df['CreateDate'])

# Ordenar por identificador de ticket y fecha de actualización
df = df.sort_values(by=['ShortId', 'LastUpdatedDate'])

# Inicializar columnas
df['TimeInSameStatus'] = 0
df['Age'] = (date_datetime - df['CreateDate']).dt.days
df['IsLastStatus'] = False

In [ ]:
#Bloque 2: Marcar último estado por ticket
for short_id, group in df.groupby('ShortId'):
    last_index = group.index[-1]
    df.at[last_index, 'IsLastStatus'] = True


In [ ]:
#Bloque 3: Calcular tiempo en la misma fase por ticket

df = df.sort_values(by=['ShortId', 'LastUpdatedDate'])
df['TimeInSameStatus'] = None

for short_id, group in df.groupby('ShortId'):
    last_status = group['Status'].iloc[-1]

    if last_status == 'Resolved':
        first_date = group['LastUpdatedDate'].iloc[0]
        last_date = group['LastUpdatedDate'].iloc[-1]
        last_status_diff = (last_date - first_date).days
        df.loc[df['ShortId'] == short_id, 'TimeInSameStatus'] = last_status_diff
        df.loc[df['ShortId'] == short_id, 'Status'] = 'Resolved'
    else:
        list_date = []
        list_status = []
        for step in group['NextStepAction'].unique():
            filtered_df_status = group[group['NextStepAction'] == step]
            min_date_status = filtered_df_status['LastUpdatedDate'].min()
            list_date.append(pd.to_datetime(min_date_status))
            list_status.append(step)

        time_diffs = []
        for i in range(1, len(list_date)):
            time_diff = (list_date[i] - list_date[i-1]).days
            time_diffs.append((list_status[i-1], time_diff))

        if list_date:
            if len(list_date) == 1:
                last_status_diff = (date_datetime - list_date[0]).days
                df.loc[(df['ShortId'] == short_id) & 
                       (df['NextStepAction'] == list_status[0]), 
                       'TimeInSameStatus'] = last_status_diff
            else:
                last_status_diff = (date_datetime - list_date[-1]).days
                df.loc[(df['ShortId'] == short_id) & 
                       (df['NextStepAction'] == list_status[-1]), 
                       'TimeInSameStatus'] = last_status_diff

        for status, time_diff in time_diffs:
            df.loc[(df['ShortId'] == short_id) & 
                   (df['NextStepAction'] == status), 
                   'TimeInSameStatus'] = time_diff


In [ ]:
#Bloque 4: Asegurar valores positivos y sin nulos
df['TimeInSameStatus'] = df['TimeInSameStatus'].fillna(0).abs()


In [ ]:
import boto3
from botocore.exceptions import ClientError

# Funciones para preservar y reaplicar ACLs
def preserve_acl_before_upload(bucket, key):
    s3 = boto3.client('s3')
    try:
        return s3.get_object_acl(Bucket=bucket, Key=key)
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchKey':
            return None
        else:
            raise
def reapply_acl_after_upload(bucket, key, acl):
    s3 = boto3.client('s3')
    if acl:
        s3.put_object_acl(
            Bucket=bucket,
            Key=key,
            AccessControlPolicy={
                'Grants': acl['Grants'],
                'Owner': acl['Owner']
            }
        )
def upload_file_to_s3_with_acl(file_path, bucket_name, key):
    s3 = boto3.client('s3')
    acl = preserve_acl_before_upload(bucket_name, key)
    s3.upload_file(file_path, bucket_name, key)
    reapply_acl_after_upload(bucket_name, key, acl)
    print(f"Archivo {file_path} subido a s3://{bucket_name}/{key} con ACL preservado.")


# Subir archivos csv a S3 mantieniendo ACL:
file="W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/"

upload_file_to_s3_with_acl(
    file_path = f"{file}Raw_Data/Merged_Data/Data_Merged_with_Calculations.csv",
    bucket_name="voiceofvehicle",
    key="dashboardvcm/Data_Merged_with_Calculations.csv"
)

upload_file_to_s3_with_acl(
    file_path = f"{file}Weekly_KPI/VCM_Metrics/Merged/VCM_Metrics_Actual.csv",
    bucket_name="voiceofvehicle",
    key="dashboardvcm/VCM_Metrics_Actual.csv"
)

upload_file_to_s3_with_acl(
    file_path= f"{file}Weekly_KPI/VCM_Metrics/Goal_Metric/Merged/VCM_Goal_Metric_Merged.csv",
    bucket_name="voiceofvehicle",
    key="dashboardvcm/VCM_Goal_Metric_Merged.csv"
)